# Author: Uday Routhu

# course of this project:

#Part 1: Download several files from IMDB’s movie data set and filter out the subset of moves requested by the stakeholder.


# Imports 

In [1]:
import os
import pandas as pd
import numpy as np

In [5]:
os.makedirs('Data/',exist_ok=True) # Confirm folder created
os.listdir("Data/")

[]

#  Data Set URL's

In [3]:
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
title_url="https://datasets.imdbws.com/title.ratings.tsv.gz"
aka_url="input\\title-akas-us-only.csv"

# Load Data sets

In [4]:
# Load the basics data set
title_basics = pd.read_csv(basics_url, sep='\t', low_memory=False)#

In [6]:
# Load the title_ratings data set
title_ratings = pd.read_csv(title_url, sep='\t', low_memory=False)

In [7]:
title_akas = pd.read_csv(aka_url)

C:\Users\uurou\AppData\Local\Temp\ipykernel_17076\2351655682.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  title_akas = pd.read_csv(aka_url)


# Title Basics Filtering/Cleaning Steps

In [8]:
# Filter 'title_basics' based on 'title_akas' by tconst and region.
us_movies = title_akas[title_akas['region'] == 'US']['titleId']
title_basics = title_basics[title_basics['tconst'].isin(us_movies)]

In [9]:
# Step 2: Replace "\N" with np.nan
title_basics = title_basics.replace({'\\N':np.nan})

In [10]:
# Step 3: Eliminate movies that are null for runtimeMinutes
title_basics = title_basics.dropna(subset=['runtimeMinutes'])

In [11]:
# Step 4: Eliminate movies that are null for genre
title_basics = title_basics.dropna(subset=['genres'])

In [12]:
# Step 5: Keep only titleType==Movie
title_basics = title_basics[title_basics['titleType'] == 'movie']

In [13]:
# Step 6: Convert the startYear column to float data type
title_basics['startYear'] = title_basics['startYear'].astype(float)

In [14]:
# Step 7: Filter the dataframe using startYear. Keep years between 2000-2021 (Including 2000 and 2021)
title_basics = title_basics[(title_basics['startYear'] >= 2000) & (title_basics['startYear'] <= 2021)]

In [15]:
# Step 8: Eliminate movies that include "Documentary" in the genre
is_documentary = title_basics['genres'].str.contains('documentary', case=False)
title_basics = title_basics[~is_documentary]

In [16]:
title_basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81891 entries, 34800 to 10174029
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          81891 non-null  object 
 1   titleType       81891 non-null  object 
 2   primaryTitle    81891 non-null  object 
 3   originalTitle   81891 non-null  object 
 4   isAdult         81891 non-null  object 
 5   startYear       81891 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  81891 non-null  object 
 8   genres          81891 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.2+ MB


In [17]:
## Save current dataframe to file.
title_basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

# Title Rating Filtering/Cleaning Steps

In [18]:
# Step 1: Filter title_ratings based on the 'tconst' values in title_basics
# Extract the 'tconst' values from the final 'title_basics' DataFrame
valid_tconst_values = title_basics['tconst']

In [19]:
# Filter 'title_ratings' based on the 'tconst' values
title_ratings = title_ratings[title_ratings['tconst'].isin(valid_tconst_values)]

In [20]:
# Step 2: Replace "\N" with np.nan (if any)
title_ratings = title_ratings.replace({'\\N':np.nan})

In [21]:
title_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68089 entries, 17918 to 1350848
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         68089 non-null  object 
 1   averageRating  68089 non-null  float64
 2   numVotes       68089 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 2.1+ MB


In [22]:
## Save current dataframe to file.
title_ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

# Title Akas Filtering/Cleaning Steps

In [23]:
# Load the title_akas data set
title_akas = pd.read_csv(aka_url)

C:\Users\uurou\AppData\Local\Temp\ipykernel_17076\3000111895.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  title_akas = pd.read_csv(aka_url)


In [24]:
# Filter the DataFrame to keep only rows where the 'region' column is 'US'.
title_akas = title_akas[title_akas['region'] == 'US']

In [25]:
# Replace all occurrences of "\N" in the DataFrame with np.nan.
title_akas = title_akas.replace({'\\N':np.nan}) 

In [26]:
title_akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1452564 entries, 0 to 1452563
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1452564 non-null  object
 1   ordering         1452564 non-null  int64 
 2   title            1452564 non-null  object
 3   region           1452564 non-null  object
 4   language         4018 non-null     object
 5   types            981678 non-null   object
 6   attributes       47016 non-null    object
 7   isOriginalTitle  1451222 non-null  object
dtypes: int64(1), object(7)
memory usage: 99.7+ MB


In [27]:
## Save current dataframe to file.
title_akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)